<h1>Creation of phenopackets from tabular data (individuals in columns)</h1>
<p>We will process <a href="https://pubmed.ncbi.nlm.nih.gov/34521999/" target="__blank">`Dingemans, et al. (2022) Establishing the phenotypic spectrum of ZTTK syndrome by analysis of 52 individuals with variants in SON</a></p>
<p>pyphetools provides a convenient way of extracting HPO terms from typical tables presented in supplemental material. Typical tables can have the individuals in columns or rows. In this case, we extract data from TABLE.</p>
<p>This note shows how to work through the table and set up the pyphetools encoder. The table was not originally available in the table, but constructed using the data in the publication</p>
<p>Users can work on one column at a time and then generate a collection of <a href="https://pubmed.ncbi.nlm.nih.gov/35705716/" target="__blank">GA4GH phenopackets</a> to represent each patient included in the original supplemental material. These phenopackets can then be used for a variety of downstream applications.</p>

In [1]:
import phenopackets as php
from google.protobuf.json_format import MessageToDict, MessageToJson
from google.protobuf.json_format import Parse, ParseDict
import pandas as pd
pd.set_option('display.max_colwidth', None) # show entire column contents, important!
from collections import defaultdict
import os
import sys
sys.path.insert(0, os.path.abspath('../../../pyphetools'))
sys.path.insert(0, os.path.abspath('../../pyphetools'))
from pyphetools.creation import *
from pyphetools.creation.simple_column_mapper import try_mapping_columns, get_separate_hpos_from_df
import numpy as np

ImportError: cannot import name 'get_separate_hpos_from_df' from 'pyphetools.creation.simple_column_mapper' (C:\Users\z184215\Documents\H_schijf_Lex\Projecten\HPO_clustering_Peter\Scripts\pyphetools\pyphetools\creation\simple_column_mapper.py)

<h2>Importing HPO data</h2>
<p>pyphetools uses the Human Phenotype Ontology (HPO) to encode phenotypic features. The recommended way of doing this is to ingest the hp.json file using HpoParser, which in turn creates an HpoConceptRecognizer object. </p>
<p>The HpoParser can accept a hpo_json_file argument if you want to use a specific file. If the argument is not passed, it will download the latext hp.json file from the HPO GitHub site and store it in a new subdirectory called hpo_data. It will not download the file if the file is already downloaded.</p>

In [ ]:
parser = HpoParser()
hpo_cr = parser.get_hpo_concept_recognizer()
hpo_version = parser.get_version()
metadata = MetaData(created_by="ORCID:0000-0002-5648-2155")
metadata.default_versions_with_hpo(version=hpo_version)

<h2>Importing the supplemental table</h2>
<p>Here, we use the pandas library to import this file (note that the Python package called openpyxl must be installed to read Excel files with pandas, although the library does not need to be imported in this notebook). pyphetools expects a pandas DataFrame as input, and users can choose any input format available for pandas include CSV, TSV, and Excel, or can use any other method to transform their input data into a Pandas DataFrame before using pyphetools.</p>

In [ ]:
df = pd.read_excel('../../data/SON/PMID_34521999.xlsx')

In [ ]:
df

<h1>Converting to row-based format</h1>
<p>To use pyphetools, we need to have the individuals represented as rows (one row per individual) and have the items of interest be encoded as column names. The required transformations for doing this may be different for different input data, but often we will want to transpose the table (using the pandas <tt>transpose</tt> function) and set the column names of the new table to the zero-th row. After this, we drop the zero-th row (otherwise, it will be interpreted as an individual by the pyphetools code).</p>
<p>After this step is completed, the remaining steps to create phenopackets are the same as in the 
    <a href="http://localhost:8888/notebooks/notebooks/Create%20phenopackets%20from%20tabular%20data%20with%20individuals%20in%20rows.ipynb" target="__blank">row-based notebook</a>.</p>
    
Furthermore, for this specific case, there is a Count features row that we want dropped, so we filter out any row that does not have Patient in the first column.

In [ ]:
dft = df.transpose()
dft.columns = dft.iloc[0]
dft.drop(dft.index[0], inplace=True)
dft.head()

Some column names might include spaces in front or after, and a couple of columns are subheadings and only contain NaNs, so lets correct that:

In [ ]:
dft.columns = dft.columns.str.strip()
dft = dft.dropna(axis=1, how='all')
dft['patient_id'] = dft.index
dft

<h2>Column mappers</h2>
<p>Please see the notebook "Create phenopackets from tabular data with individuals in rows" for explanations. In the following cell we create a dictionary for the ColumnMappers. Note that the code is identical except that we use the df.loc function to get the corresponding row data</p>

In [ ]:
hpo_cr = parser.get_hpo_concept_recognizer()
column_mapper_d, col_not_found = try_mapping_columns(df=dft,
                                                    observed='+',
                                                    excluded='-',
                                                    hpo_cr=hpo_cr,
                                                    preview=True)

In [ ]:
print(col_not_found)

In [ ]:
headcircumference = {'> P98': 'Macrocephaly at birth',
                 '< P3': 'Primary microcephaly',
        }
headcircumferenceMapper = OptionColumnMapper(concept_recognizer=hpo_cr, option_d=headcircumference)
print(headcircumferenceMapper.preview_column(dft['Head circumference (at birth) (HP:0011451 / HP:0004488)']))
column_mapper_d['Head circumference (at birth) (HP:0011451 / HP:0004488)'] = headcircumferenceMapper

headcircumference = {'> P98': 'Macrocephaly',
                 '< P3': 'Microcephaly',
        }
headcircumferenceMapper = OptionColumnMapper(concept_recognizer=hpo_cr, option_d=headcircumference)
print(headcircumferenceMapper.preview_column(dft['Head circumference (HP:0000252 / HP:0000256)']))
column_mapper_d['Head circumference (HP:0000252 / HP:0000256)'] = headcircumferenceMapper

In [ ]:
birth_length = {'> P98': 'Birth length greater than 97th percentile',
                 '< P3': 'Birth length less than 3rd percentile',
        }
birth_lengthMapper = OptionColumnMapper(concept_recognizer=hpo_cr, option_d=birth_length)
print(birth_lengthMapper.preview_column(dft['Heigth (at birth) (HP:0003561 / HP:0003517)']))
column_mapper_d['Heigth (at birth) (HP:0003561 / HP:0003517)'] = birth_lengthMapper

length = {'> P98': 'Tall stature',
                 '< P3': 'Short stature',
        }
lengthMapper = OptionColumnMapper(concept_recognizer=hpo_cr, option_d=length)
print(lengthMapper.preview_column(dft['Heigth (HP:0004322 / HP:0000098)']))
column_mapper_d['Heigth (HP:0004322 / HP:0000098)'] = lengthMapper

In [ ]:
birth_weight = {'> P98': 'Large for gestational age',
                 '< P3': 'Small for gestational age',
        }
birth_weightMapper = OptionColumnMapper(concept_recognizer=hpo_cr, option_d=birth_weight)
print(birth_weightMapper.preview_column(dft['Weigth (at birth) (HP:0001518 / HP:0001520)']))
column_mapper_d['Weigth (at birth) (HP:0001518 / HP:0001520)'] = birth_weightMapper

weight = {'> P98': 'Increased body weight',
                 '< P3': 'Decreased body weight',
        }
weightMapper = OptionColumnMapper(concept_recognizer=hpo_cr, option_d=weight)
print(weightMapper.preview_column(dft['Weigth (HP:0004325 / HP:0004324)']))
column_mapper_d['Weigth (HP:0004325 / HP:0004324)'] = weightMapper

In [ ]:
id_severity = {'Mild': 'Intellectual disability, mild',
                 'Moderate': 'Intellectual disability, moderate',
         'Severe': 'Intellectual disability, severe'
        }
id_severityMapper = OptionColumnMapper(concept_recognizer=hpo_cr, option_d=id_severity)
print(id_severityMapper.preview_column(dft['Severity of intellectual disability (HP:0001256 / HP:0002342 / HP:0010864)']))
column_mapper_d['Severity of intellectual disability (HP:0001256 / HP:0002342 / HP:0010864)'] = id_severityMapper

For this particular file, there are HPO terms in the cells of the table as well, so we should loop them, parse contents and add them to the parser.

In [ ]:
additional_hpos = get_additional_hpos_from_df(dft, hpo_cr)

<h2>Variant Data</h2>
<p>The variant data (HGVS< transcript) is listed in the Variant (hg19, NM_015133.4) column.</p>

In [ ]:
genome = 'hg38'
default_genotype = 'heterozygous'
transcript='NM_138927.2'
varMapper = VariantColumnMapper(assembly=genome,column_name='cDNA change', 
                                transcript=transcript, genotype=default_genotype)

<h1>Demographic data</h1>
<p>pyphetools can be used to capture information about age, sex, and individual identifiers. This information is stored in a map of "IndividualMapper" objects. Special treatment may be required for the indifiers, which may be used as the column names or row index.</p>

In [ ]:
ageMapper = AgeColumnMapper.by_year('Age at examination')
ageMapper.preview_column(dft['Age at examination'])

In [ ]:
sexMapper = SexColumnMapper(male_symbol='Male', female_symbol='Female', column_name='Gender')
sexMapper.preview_column(dft['Gender'])

In [ ]:
pmid = "PMID: 34521999"
encoder = CohortEncoder(df=dft, hpo_cr=hpo_cr, column_mapper_d=column_mapper_d, 
                        individual_column_name="patient_id", agemapper=ageMapper, sexmapper=sexMapper,
                       variant_mapper=varMapper, metadata=metadata,
                       pmid=pmid)
encoder.set_disease(disease_id='617140', label='ZTTK SYNDROME')

In [ ]:
individuals = encoder.get_individuals(additional_hpos)

In [ ]:
i1 = individuals[0]
phenopacket1 = i1.to_ga4gh_phenopacket(metadata=metadata.to_ga4gh())
json_string = MessageToJson(phenopacket1)
print(json_string)

In [ ]:
output_directory = "../../phenopackets/SON/"
encoder.output_phenopackets(outdir=output_directory)